In [1]:
import psycopg2 as psy
from psycopg2 import sql

#from main import show_img, plot_graph, predict

from config import config

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras

import matplotlib.pyplot as plt
import cv2
import os

2025-01-10 16:20:53.276802: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-10 16:20:53.289301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736526053.301919   45832 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736526053.305276   45832 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-10 16:20:53.318977: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model = tf.keras.models.load_model('savedModels/test_model.keras')


I0000 00:00:1736526056.839600   45832 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5465 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
def connectTest():
    

    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psy.connect(**config()) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psy.DatabaseError, Exception) as error:
        print(error)

connectTest()

Connected to the PostgreSQL server.


<connection object at 0x7f3af591f600; dsn: 'user=postgres password=xxx host=localhost port=5430', closed: 0>

In [ ]:
def connect_Create_db():
    conn = None
    try:
        conn = psy.connect(**config())
        conn.autocommit = True
        print("Connected to the PostgreSQL server")

        crsr = conn.cursor()
        crsr.execute('SELECT version()')
        db_version = crsr.fetchone()
        #crsr.close()
        print(f"PostgreSQL database version: {db_version}")

        dbname = "pycreatetest2"
        #crsr.execute('CREATE DATABASE pycreatetest ;')
        #crsr.execute('SELECT datname FROM pg_database WHERE datistemplate = false;')
        crsr.execute(f"SELECT datname FROM pg_database WHERE datname = '{dbname}'")
        tables = crsr.fetchall()
        #print(len(tables))
        if len(tables) == 0:
            crsr.execute(f'CREATE DATABASE {dbname}')
        crsr.close()

    except(Exception, psy.DatabaseError) as error:
        print(error)

    finally:
        if conn != None:
            conn.close()
            print("Connection closed")

connect_Create_db()


Connected to the PostgreSQL server
PostgreSQL database version: ('PostgreSQL 16.6 (Debian 16.6-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
Connection closed


In [199]:
def create_table():
    '''
    Create postgres table to store images and data
    '''
    try:
        conn = psy.connect(**config(),database='idtest') #Hardcoded database choice, can be moved to ini later
        conn.autocommit = True

        crsr = conn.cursor()
        crsr.execute('''CREATE TABLE dummy (id INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY, image TEXT, label INT, prediction INT, correct BOOL, certainty REAL);''')
        
    except(Exception, psy.DatabaseError) as error:
        print (error)

    finally:
        crsr.close()
        conn.close()

create_table()

relation "dummy" already exists



In [75]:
def connect():
    try:
        conn = psy.connect(**config(),database='testdb')
        print(conn)
        conn.autocommit = True
        crsr=conn.cursor()
        #crsr.execute("CREATE TABLE tableTest (ID INT, Num INT)")
        #crsr.execute("INSERT INTO tableTest VALUES (1,2);")
        #print(crsr.fetchall())
        crsr.execute("SELECT * FROM tableTest")
        print(crsr.fetchall())
        crsr.close()
    except(Exception, psy.DatabaseError) as error:
        print (error)
    finally:
        if conn != None:
            conn.close()
            print("Connection closed")

connect()


<connection object at 0x7f4a3746f740; dsn: 'user=postgres password=xxx dbname=testdb host=localhost port=5430', closed: 0>
[(1, 2), (1, 2), (1, 2)]
Connection closed


In [173]:
def load_to_postgres(path:str="numbers"):
    '''
    Load dataset from folder into postgresql table
    
    Parameters
    ----------
    
    path: (str) path to root folder containing dataset

    Returns
    ----------
    
    numbers: (ndarray) (n,24,24) Array of number images
    
    labels: (ndarray) (n,) Array of labels
    
    Folder structure
    -----------------
    
    /numbers #root dir
        numbers/0
            img.jpg
            img.jpg
            .
            .
            .
        numbers/1   
            img.jpg
            img.jpg
            .
            .
            .
        .
        .
        .
        
    numbers folder containing folders with folder name as the label (0 to 9)
    Image names within label folders not relevant
    '''
    try:
        conn = psy.connect(**config(),database='idtest')
        print(conn)
        conn.autocommit = True
        crsr=conn.cursor()
        for i in range(10):
            for dirpath,dirnames,filenames in (os.walk(f"{path}/{i}")):
                for file in filenames:
                    img = cv2.imread(f"{path}/{i}/{file}",cv2.IMREAD_GRAYSCALE) 
                    img = cv2.resize(img,(28,28), interpolation=cv2.INTER_AREA)
                    img = (255-img)/255.0
                    #img = np.reshape(img,(1,28,28))

                    temp_image = np.reshape(img,(-1)) #Image stored as a string (TEXT) of a 1D array in postgres
                    
                    crsr.execute(f'''INSERT INTO dummy(image, label) VALUES ('{temp_image}',{i})''')
        crsr.close()
    except(Exception, psy.DatabaseError) as error:
        print (error)
    finally:
        if conn != None:
            conn.close()
            print("Connection closed")
    
load_to_postgres()

<connection object at 0x7f49b015fd80; dsn: 'user=postgres password=xxx dbname=idtest host=localhost port=5430', closed: 0>
Connection closed


In [164]:
def show_img(i, data, labels):
    '''
    Show image at index i of array of images
    '''
    plt.imshow((data)[i], cmap=plt.cm.binary)
    
    predA = (model.predict(data)[i])
    pred = np.argmax(predA)
    certainty = round((np.max(predA)*100),2)

    if (pred == labels[i]):
        color = 'green'
    else:
        color = 'red'
        
    plt.xlabel(f"Pred:{pred} (Actual:{labels[i]})", color=color)
    plt.show()

    return(pred, certainty)

In [ ]:
def predict_from_postgres(id):
    '''
    Predict number & uncertainty, display processed image, from postgres table

    Parameters
    ----------
    id (int): Primary key from postgres table
    '''
    try:
        conn = psy.connect(**config(),database='idtest') #Hardcoded database choice, can be moved to ini later
        conn.autocommit = True

        crsr = conn.cursor()
        crsr.execute(f'''SELECT image FROM dummy WHERE id = {id}''')
        data = str(crsr.fetchone()[0])

        crsr.execute(f'''SELECT label FROM dummy WHERE id = {id}''')
        label = crsr.fetchone()[0]

        #Image stored as a string (TEXT) of a 1D array in postgres convert to a np.ndarray
        img = (np.fromstring(data.strip('[]'),dtype=float, sep=' '))

        img = np.reshape(img, (1,28,28)) #Reshape image as model.predict requires 3D array

        prediction, certainty = show_img(0,img,[label])

        if (prediction != None) & (prediction == label):
            correct = True
        elif (prediction != label):
            correct = False
        
        #Update table values
        crsr.execute(f'''UPDATE dummy SET prediction = {prediction}, certainty = {certainty}, correct = {correct} WHERE id = {id}''')

        

    except(Exception, psy.DatabaseError) as error:
        print (error)

    finally:
        crsr.close()
        conn.close()

for i in range(1,13):
    predict_from_postgres(i)


In [8]:
#print sorted list of ids where predictions are correct

import operator
conn = psy.connect(**config(),database='idtest') #Hardcoded database choice, can be moved to ini later
conn.autocommit = True



crsr = conn.cursor()

crsr.execute('''SELECT id FROM dummy WHERE correct IS NULL OR label IS NULL''')
x = crsr.fetchall()
crsr.close()
conn.close()
print(sorted(list(map(operator.itemgetter(0), x))))

[19]
